In [ ]:
!apt-get update && apt-get install ffmpeg tmux vim -y
!pip3 install moviepy openai python-dotenv pydub pytubefix openai-whisper llama-index llama-index-llms-openai llama-index-llms-ollama llama-index-embeddings-ollama 
!pip3 install flash-attn --no-build-isolation
!curl -fsSL https://ollama.com/install.sh | sh

# !chmod +x /usr/bin/ollama
# !useradd -r -s /bin/false -m -d /usr/share/ollama ollama
# !pip3 install openpyxl sentencepiece protobuf evaluate rouge_score absl-py tensorboardX bitsandbytes peft accelerate python-dotenv dspy-ai graspologic


In [ ]:
import os, pickle, gc, subprocess
from dotenv import load_dotenv
from video_transcription import split_audio_into_chunks, video_to_audio, transcribe_audio_chunks, get_transcription_model, download_video
from llama_index.core import Document
# import torch

load_dotenv("/workspace/repos/agentic-ai/.env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

video_path="/workspace/data/video1512218125.mp4"
# video_path="https://www.youtube.com/watch?v=Z07Ewop7rQA"
audio_output_path="/workspace/data/video_audio.mp3"
transcribe_output_dir="/workspace/data"
file_save_path="/workspace/data/transcription.pkl"

In [ ]:
if not os.path.exists(file_save_path):
    print("Loading transcription model...")
    transcribe_model = get_transcription_model(open_source_model=True)

    print("Processing video...")
    if 'youtube' in video_path:
        print("   Downloading video from youtube...")
        download_video(video_url=video_path, audio_output_path=audio_output_path)
    else:
        video_to_audio(video_path=video_path, audio_output_path=audio_output_path)

    print("Splitting audio into chunks...")
    split_audio_into_chunks(audio_output_path=audio_output_path, transcribe_output_dir=transcribe_output_dir, max_chunk_size_mb=24)

    print("Transcribing audio chunks...")
    documents = transcribe_audio_chunks(model=transcribe_model, chunk_dir="/workspace/data", file_save_path=file_save_path)

    del transcribe_model
    # torch.cuda.empty_cache()
    gc.collect()
else:
    with open(file_save_path, 'rb') as f:
        documents = pickle.load(f)
    
if isinstance(documents[0], dict):
    documents = [Document(text=chunk["text"]) for chunk in documents]

full_doc = " ".join([doc.text for doc in documents])


In [ ]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI as LOpenAI

# model_name, ctx_len = "gpt-4o", 128000
# model_name, ctx_len = "llama3.1:latest", 128000
model_name, ctx_len = "hermes3:8b", 128000

if model_name == "gpt-4o":
    llm = LOpenAI(model=model_name, max_tokens=4000)
else:
    try: 
        print("Pulling Ollama model...")
        sub_out = subprocess.run(['ollama', 'pull', model_name], capture_output=True, text=True)
    except Exception as e: 
        print(f"Error pulling model: Is the Ollama server running?\n{e}")
    
    addtion_kwargs = {"max_new_tokens": 4000}
    # system_prompt = "You are an expert at answering questions about rules and regulations regarding Title 17—Commodity and Securities Exchanges: CHAPTER II—SECURITIES AND EXCHANGE COMMISSION. Please provide a summary of the following text, and cite any sections, rules, acts or laws (e.g. § 230.503, § 240.13a-15, Act (15 U.S.C. 781), Investment Company Act of 1940) from context that support the answer. Be detailed in your response."
    llm = Ollama(model=model_name, url="http://127.0.0.1:11434", context_window=ctx_len, model_type="chat", is_function_calling_model=False, 
                request_timeout=1000.0, additional_kwargs=addtion_kwargs) # system_prompt=system_prompt
    print(llm.metadata)

Settings.llm = llm

## Tree summarizer

In [ ]:
from llama_index.core.response_synthesizers import TreeSummarize
summarizer = TreeSummarize(llm=llm, verbose=True)
# prompt_summary = "You are a professional executive of AlphaTrAI. Your job is to summarize this text in great detail from a video transcription. The summary will be distributed to investors and stakeholders, so give a lot of details and examples from the transcription."
prompt_summary = f"""You are a professional executive at AlphaTrAI. Your job is to summarize the text from a video transcription. The summary will be a memo distributed to investors and stakeholders. Be sure it the memo has the following items:
1. Extract all the names of new hires and their position, and/or new advisors mentioned in the transcription.
2. Create a section to mention the personnel new to AlphaTrAI.
3. Include other highlights and progress made by AlphaTrAI.
4. Ensure the memo and ensure it is factual, optimistic, and any values mention come directly from the text. 

The transcription is as follows:\n{full_doc}"""

response = await summarizer.aget_response(prompt_summary, [doc.text for doc in documents])

In [ ]:
print(response)

## LLM direct summarization

In [ ]:
prompt_summary = f"""You are a professional executive at AlphaTrAI. Your job is to summarize the text from a video transcription. The summary will be a memo distributed to investors and stakeholders. Be sure it the memo has the following items:
1. Extract all the names of new hires and their position, and/or new advisors mentioned in the transcription.
2. Include other highlights and progress made by AlphaTrAI.
3. Ensure the memo is professional, fluid, factual, and optimistic. 

The transcription is as follows:\n{full_doc}"""

response = llm.complete(prompt_summary, max_tokens=5000)

In [ ]:
print(response.text)

## Agentic Summary

In [ ]:
!pip3 install llama-index-embeddings-huggingface llama-index-vector-stores-neo4jvector llama-index-graph-stores-neo4j
!apt install dialog apt-utils -y (done above)
!wget -O - https://debian.neo4j.com/neotechnology.gpg.key | gpg --dearmor -o /etc/apt/keyrings/neotechnology.gpg
!echo 'deb [signed-by=/etc/apt/keyrings/neotechnology.gpg] https://debian.neo4j.com stable latest' | tee -a /etc/apt/sources.list.d/neo4j.list
!apt list -a neo4j
!add-apt-repository universe -y
!apt install neo4j=1:5.22.0 -y
!echo "neo4j-enterprise neo4j/question select I ACCEPT" | debconf-set-selections
!echo "neo4j-enterprise neo4j/license note" | debconf-set-selections
!apt install openjdk-17-jre -y
!cd /var/lib/neo4j/plugins/ && wget https://github.com/neo4j/apoc/releases/download/5.22.0/apoc-5.22.0-core.jar

In [ ]:
set_neo4j_password('bewaretheneo')
add_lines_to_conf()


In [ ]:
from dotenv import load_dotenv
load_dotenv()

from llama_index.core.agent import ReActAgent
# from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from rag_utils import create_neo4j_graph_store, create_neo4j_graphrag, neo4j_query, set_neo4j_password, add_lines_to_conf
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import nest_asyncio
nest_asyncio.apply()



In [ ]:

llm.is_function_calling_model = True

embed_model_name = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
print("loading embed model...")
embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

Settings.embed_model = embed_model
Settings.chunk_size = 300
Settings.chunk_overlap = 50

In [ ]:
from typing import Literal
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

entities = Literal["PEOPLE", 
                   "PLACE"
]

relations = Literal[
    "ROLE",
    "COMPANY"
]

validation_schema = {
    "People": ["ROLE"],
    "Place": ["COMPANY"],
}


In [ ]:
Settings.chunk_size = 300
Settings.chunk_overlap = 50

kg_extractor = SchemaLLMPathExtractor(
    llm=llm,
    possible_entities=entities,
    possible_relations=relations,
    kg_validation_schema=validation_schema,
    strict=True,  # if false, will allow triples outside of the schema
    num_workers=4,
    max_triplets_per_chunk=10,
)

graph_store = create_neo4j_graph_store(neo_url="bolt://localhost:7687", 
                                       password=os.getenv("NEO4J_PWD"), 
                                       config={"connection_timeout": 240, "connection_acquisition_timeout": 240, "max_connection_pool_size": 1000})
neo4j_query(graph_store, query="""MATCH (n) DETACH DELETE n""")


graph_index = create_neo4j_graphrag(documents, llm, embed_model, kg_extractor, graph_store)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = QueryEngineTool(
            query_engine=graph_index,
            metadata=ToolMetadata(
                name="graph_tool",
                description=(
                    "Useful for finding people names and roles, and the company they work for."
                ),
            ),
        ),


In [ ]:
!pip3 install llama-agents

In [ ]:
from llama_agents import (
    AgentService,
    ToolService,
    MetaServiceTool,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import ReActAgentWorker, ReActAgent



# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=llm),
)

# define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=[query_engine_tools],#, adding_tool],
    running=True,
    step_interval=0.5,
)

# define meta-tools here
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    )
    for t in [query_engine_tools]#, adding_tool]
]


# define Agent and agent service
# worker1 = FunctionCallingAgentWorker.from_tools(
worker1 = ReActAgentWorker.from_tools(
    meta_tools,
    llm=llm,
)
agent1 = worker1.as_agent()
agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Summarize a transcription as a memo for investors and stakeholders.",
    service_name="summarize_transcription",
)

In [ ]:
asset_list = ["Vanguard Group",	"BlackRock",	"State Street Global",	"Fidelity Investments",	"BNY Mellon",	"Goldman Sachs Group",	"J.P. Morgan Asset & Wealth",	"Legal & General Investment",	"Wellington Mgmt.",	"Amundi",	"Prudential Financial",	"Geode Capital Mgmt.",	"PIMCO",	"Northern Trust Asset Mgmt.",	"Nuveen",	"Capital Group",	"T. Rowe Price Associates",	"AXA Investment",	"Franklin Templeton",	"Federated Hermes",	"Invescos",	"Dimensional Fund Advisors",	"MetLife Investment Mgmt.",	"Morgan Stanley Inv. Mgmt.",	"New York Life Investments",	"Schroders",	"Principal Global Investors",	"KKR",	"DWS",	"Macquarie Asset Mgmt.",	"Brookfield Asset Mgmt.",	"Allspring Global Investments",	"BNP Paribas Asset Mgmt.",	"Asset Management One",	"Mercer",	"Barings",	"Aegon Asset Mgmt.",	"AllianceBernstein",	"Neuberger Berman",	"Ares Mgmt.",	"Columbia Threadneedle",	"NISA Investment",	"Voya Investment Mgmt.",	"MassMutual",	"Aviva Investors",	"SEI Investments",	"Manulife Investment",	"SLC Management",	"Russell Investments",	"Loomis, Sayles",	"Baillie Gifford Overseas",	"Dodge & Cox",	"TCW Group",	"MFS Investment",	"RBC Global Asset Mgmt.",	"Mesirow",	"Guggenheim Investments",	"Wilmington Trust",	"WTW Investment Services",	"Conning",	"Empower Investments",	"PFM Asset Mgmt.",	"CBRE Investment Mgmt.",	"TD Global Invest. Solutions",	"IFM Investors",	"Arrowstreet Capital",	"Nomura Asset Mgmt.",	"Oaktree Capital",	"Payden & Rygel",	"Lazard Asset Mgmt.",	"Victory Capital",	"Artisan Partners",	"PRIMECAP",	"Man Group",	"Robeco",	"Baird Advisors",	"Hamilton Lane",	"Ninety One",	"Partners Group",	"Starwood Capital",	"AQR Capital Mgmt.",	"Acadian Asset Mgmt.",	"Prologis",	"LSV Asset Mgmt.",	"StepStone Group",	"American Century",	"Charles Schwab Investment",	"RhumbLine Advisers",	"Pathway Capital",	"Boston Partners",	"Record Currency Mgmt.",	"Income Research & Mgmt.",	"AEW Capital",	"Pictet Asset Mgmt.",	"First Sentier Investors",	"Hines",	"New England Asset Mgmt.",	"Alan Biller and Associates",	"PPM America",	"LaSalle Investment",	"GCM Grosvenor",	"PineBridge Investments",	"PNC Financial",	"CC&L Financial Group",	"Fort Washington",	"Dai-ichi Life Holdings",	"Oak Hill Advisors",	"William Blair",	"Putnam Investments",	"Ashmore Group",	"Heitman",	"Harrison Street",	"Grantham, Mayo v. Otterloo",	"PAG",	"Harris Associates",	"Adams Street Partners",	"Sterling Capital",	"GoldenTree Asset Mgmt.",	"Mondrian Investment",	"Angelo, Gordon",	"Nikko Asset Mgmt.",	"Harding Loevner",	"Brown Advisory",	"Portfolio Advisors",	"Fisher Investments",	"Cohen & Steers",	"Marathon-London",	"Harbor Capital Advisors",	"Aristotle Capital Mgmt.",	"SECOR Asset Mgmt.",	"Stockbridge Capital Group",	"PanAgora Asset Mgmt.",	"Pzena Investment",	"Causeway Capital",	"Colchester Global Investors",	"MissionSquare Investments",	"Hayfin Capital Mgmt.",	"ORIX USA",	"CIBC Asset Mgmt.",	"Los Angeles Capital",	"Shenkman Group",	"Jarislowsky Fraser",	"EARNEST Partners",	"Knights of Columbus Asset",	"Strategic Investment Group",	"Commonfund",	"Rockpoint Group",	"Hotchkis & Wiley",	"AAM",	"CIM Group*",	"Beutel, Goodman",	"Nomura Corporate Research",	"Scout Investments",	"Calamos Advisors",	"ACORE Capital",	"PCCP",	"Guardian Capital",	"DuPont Capital",	"Canyon Partners",	"Kayne Anderson Rudnick",	"Polen Capital",	"TA Realty",	"Sustainable Growth Advisers",	"MFG Asset Mgmt.",	"Unigestion",	"Intech",	"Eagle Capital",	"Garcia Hamilton",	"Sprucegrove Investment",	"Longfellow Investment",	"Axiom Investors",	"King Street Capital",	"Wasatch Global Investors",	"Boyd Watterson",	"Champlain Investment",	"Crestline Investors",	"Callan",	"Pacific Asset Mgmt.",	"Fuller & Thaler",	"Cantillon Capital Mgmt.",	"Jacobs Levy Equity",	"Brandes Investment",	"Fayez Sarofim",	"Sit Investment",	"Cliffwater",	"Intercontinental Real Estate",	"Walton Street Capital",	"Beacon Capital",	"Rockwood Capital",	"Breckinridge Capital",	"Beach Point Capital",	"Amalgamated Bank",	"American Realty Advisors",	"Abbott Capital",	"Eagle Asset Mgmt.",	"Westfield Capital",	"Driehaus Capital",	"CenterSquare Investment",	"Segall Bryant & Hamill",	"Polaris Capital",	"Grayscale Investments",	"GW&K Investment",	"CornerStone Partners",	"Westbrook Partners",	"Bahl & Gaynor",	"Sage Advisory Services",	"Yousif Capital",	"Commerce Trust",	"Zacks Investment",	"Stone Harbor Investment",	"Brown Capital",	"Cooke & Bieler",	"L&B Realty",	"Jensen Investment",	"Burgundy Asset Mgmt.",	"Pugh Capital",	"Mill Creek Residential",	"Global Endowment Mgmt.",	"Ullico Investment",	"London Co.",	"GAMCO Investors",	"Matthews Asia",	"Capital Fund Mgmt.",	"Waterfall Asset Mgmt.",	"Westwood Global",	"Frontier Capital",	"Christian Brothers",	"Manning & Napier",	"Ariel Investments",	"Washington Capital",	"TimesSquare Capital",	"Ramirez Asset Mgmt.",	"Altrinsic Global Advisors",	"National Real Estate",	"Advent Capital",	"CS McKee",	"WEDGE Capital",	"Newfleet Asset Mgmt.",	"National Investment",	"TT International",	"Prima Capital Advisors",	"Alger",	"Silver Creek Capital",	"River Road Asset Mgmt.",	"Agincourt Capital",	"Diamond Hill Capital",	"AGF Investments",	"Sentinel Real Estate",	"Ceredex Value Advisors",	"CoreCommodity",	"LCM Partners",	"Madison Realty",	"Silvercrest Asset Mgmt.",	"White Oak Global Advisors",	"Luther King Capital",	"Equus Capital",	"Hardman Johnston Global",	"AFL-CIO Housing Trust",	"Corbin Capital",	"City of London",	"Spider Mgmt.",	"M3 Capital",	"Davis Advisors",	"Torchlight Investors",	"Stephens Inv. Mgmt. Group",	"Great Lakes Advisors",	"Congress Asset Mgmt.",	"Parnassus Investments",	"Dana Investment",	"Martingale Asset Mgmt.",	"Madison Investments",	"Richmond Capital",	"Camden Asset Mgmt.",	"400 Capital Mgmt.",	"Glenmede Investment",	"Lyrical Asset Mgmt.",	"Gramercy",	"D.F. Dent",	"Resource Mgmt.",	"DePrince, Race & Zollo",	"Fiduciary Mgmt./Milwaukee",	"Duff & Phelps",	"AFL-CIO Building Trust",	"Johnson Asset Mgmt.",	"LM Capital Group",	"Conestoga Capital",	"Sierra Investment",	"Baird Equity Asset Mgmt.",	"Forest Investment",	"Carmel Partners",	"Atalanta Sosnoff Capital",	"Jackson Square Partners",	"Peregrine Capital",	"Todd Asset Mgmt.",	"Hoisington Investment",	"GlobeFlex Capital",	"Kornitzer Capital",	"Patron Capital",	"Emerald Advisers",	"Mar Vista Investment",	"Stacey Braun Associates",	"Leading Edge Investment",	"Kennedy Capital",	"Security Capital Research",	"Riverbridge Partners",	"Cardinal Capital",	"Granahan Investment",	"Dolan McEniry",	"Angel Oak Capital",	"Global Forest Partners",	"Channing Capital",	"ClariVest Asset Mgmt.",	"Twin Bridge Capital",	"Evanston Capital",	"Aristotle Capital Boston",	"Systematic Financial",	"Palisade Capital",	"Algert Global",	"Hillsdale Investment",	"Prime Group",	"3650 REIT",	"GTIS Partners",	"Bivium Capital",	"Molpus Woodlands Group",	"Winthrop Capital",	"Zevenbergen Capital",	"Dalton Investments",	"Hood River Capital",	"Trillium Asset Mgmt.",	"Bowen, Hanes",	"EAM Investors",	"Verger Capital",	"GIA Partners",	"Weatherbie Capital",	"Oberweis Asset Mgmt.",	"Zazove Associates",	"Wexford Capital",	"Timberland Inv. Resources",	"Sawgrass Asset Mgmt.",	"SSI Investment",	"NewSouth Capital",	"Foundry Partners",	"Sound Shore Mgmt.",	"KBS",	"HS Management",	"Silvant Capital",	"Millstreet Capital",	"Redwood Investments",	"Genter Capital",	"Smith Group Asset Mgmt.",	"Sarofim Realty",	"Edgar Lomax",	"Covenant Capital Group",	"Heartland Advisors",	"Adelante Capital",	"Rice Hall James",	"Cramer Rosenthal McGlynn",	"Quest Investment",	"Penn Capital",	"Ranger Investments",	"Associated Capital Group",	"Cornerstone Investment",	"Smith Affiliated Capital",	"Logan Capital",	"Wilbanks, Smith & Thomas",	"Orleans Capital",	"Thornburg Investment",	"Karpus Investment",	"StoneRidge PMG Advisors",	"Tributary Capital",	"Mairs & Power",	"Bridgeway Capital",	"Granite Investment",	"Ativo Capital Mgmt.",	"Nicholas Investment",	"Sasco Capital",	"CS Capital",	"TWIN Capital",	"CI Inverness",	"Miller/Howard Investments",	"Belle Haven Investments",	"Montag & Caldwell",	"Anchor Capital",	"Wedgewood Partners",	"Wright Investors' Service",	"Phocas Financial",	"TSCG Investors",	"Pier Capital",	"GLOBALT",	"Van Hulzen Asset Mgmt.",	"SKBA Capital Mgmt.",	"Domain Timber Advisors",	"Speece Thorson Capital",	"Redstone Advisors",	"Aristotle Credit Partners",	"TerraCotta Group",	"Farr, Miller & Washington",	"SouthernSun Asset Mgmt.",	"Gifford Fong Associates",	"Denali Advisors",	"KDP Asset Mgmt.",	"AMI Asset Mgmt.",	"Semper Capital",	"Renaissance Investment",	"ZWJ Investment Counsel",	"Campbell Newman Asset",	"Gateway Investment",	"SMH Capital Advisors",	"Argent Capital",	"Chicago Capital",	"Osborne Partners",	"Oak Associates",	"Windham Capital",	"Bridge City",	"Strategy Asset Managers",	"Kingdon Capital",	"Glovista Investments",	"Winslow Asset Mgmt.",	"Hahn Capital",	"Affinity Investment Advisors",	"Teton Advisors",	"Abner, Herrman & Brock",	"NovaPoint Capital",	"Paradigm Capital",	"Flippin, Bruce & Porter",	"Kestrel Investment",	"Tom Johnson Investment",	"Argus Investors' Counsel",	"Branson, Fowlkes/Russell",	"Robinson Value Mgmt.",	"Chase Investment Counsel",	"Nicholas Co.",	"Cadinha",	"Pacific West Land",]
import os, re
from dotenv import load_dotenv
load_dotenv('/workspace/repos/agentic-ai/.env')
from llama_index.llms.openai import OpenAI as LOpenAI

model_name, ctx_len = "gpt-4o-2024-08-06", 128000
openai_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key
system_prompt = f"""You are gathering information on the top asset management firms in the world. 
This information will be put in a spreadsheet. 
You need to know the CEO of each firm.
Return the company name and CEO name in the following format: "Company: CEO".
"""
print(f"Using OpenAI {model_name}...")
llm = LOpenAI(model=model_name, max_tokens=8000, system_prompt=system_prompt)
ceos=[]
coos=[]
cios=[]
presidents=[]
execs = [ceos, coos, cios, presidents]
exec_strs = ["Chief Executive Officer", "Chief Operating Officer", "Chief Investment Officer", "President"]
results = []

for i,a in enumerate(asset_list):
    company_execs=[a]
    for exec_type, exec_str in zip(execs, exec_strs):
        system_prompt = f"""You are gathering information on the executives at top asset management firms. 
        Only output the executive's name. 
        Here are examples of properly formatted outputs: 'Larry Fink', 'Lori Heinel', 'James S. Anderson', 'Brian Carlin', 'Michael Sacks').
        
        Who is the {exec_str} of {a}?
        """
        response = llm.complete(system_prompt)
        # response_check = llm.complete(f"Extract and only output the name in the following text.\n\n{response.text}")
        company_execs.append(response.text)
        
    # if i==10:
    #     raise ValueError("Stop here.")
    # else:
    #     print(company_execs)
    if i%50==0:
        print(f"{i} {company_execs}...")
    results.append(company_execs)
    
header=[["Company Name"]+["Chief Executive Officer", "Chief Operating Officer", "Chief Investment Officer", "President"]]
bads=0
for r in results:
    for elem in r:
        if len(elem.split())>4:
            bads+=1
            print(" ".join(elem.split()[-3:]))
            break
print(bads)
results=[r[0] for r in results]
for i in range(len(results)):
    for j in range(1,5):
        # results[i][j] = results[i][j].strip().replace("'", "")
        # if results[i][j].endswith("."):
        #     results[i][j] = results[i][j][:-1]
        # if results[i][j].endswith(","):
        #     results[i][j] = results[i][j][:-1]
        # if " ".join(results[i][j].split()[-3:]).startswith("is"):
        #     results[i][j] = " ".join(results[i][j].split()[-3:])[2:].strip()
        # if " ".join(results[i][j].split()[-3:]).startswith(("is", "have", "and", "a different", "my")):
        #     results[i][j] = " ".join(results[i][j].split()[-3:])[2:].strip()
        if len(results[i][j].split())>4:
            results[i][j] = " ".join(results[i][j].split()[-3:])[2:].replace('"', '').strip()
import csv

# Sample list of lists


# Specify the file name
filename = '/workspace/data/asset_execs.csv'

# Open the file in write mode
with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Write each row to the CSV file
    for row in header+results:
        writer.writerow(row)

print(f"Data has been written to {filename}")